In [27]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
import numpy as np
import random
import json
from pprint import pprint

In [2]:
#keys for apis
with open('C:/Users/james/.secret/patronOmdb.txt', 'r') as f:
    omdb_key = f.read()

with open('C:/Users/james/.secret/imdbApi.txt', 'r') as g:
    imdb_key = g.read()

with open('C:/Users/james/.secret/tmdbApi.txt', 'r') as h:
    tmdb_key = h.read()  


In [36]:
#open up all csvs as dataframes

df = pd.read_csv('df.csv')
financial_df = pd.read_csv('financial_df.csv')
non_english_df = pd.read_csv('non_english_df.csv')

with open('omdb_attrs.json') as json_file:
    omdb_attrs = json.load(json_file)

attributes_df = pd.read_csv('attributes_df.csv')

In [37]:
attributes_df.head(1)

,imdb_code,Title,Actors,Director,Genre,Plot,Rated,Runtime,Writer,RottenRating,MetacriticRating,imdbRating,imdbVotes
0,tt5433138,F9,"Vin Diesel, Michelle Rodriguez, Jordana Brewster",Justin Lin,"Action, Adventure, Crime",Dom and the crew must take on an international...,PG-13,143 min,"Daniel Casey, Justin Lin, Alfredo Botello",59.0,58.0,5.2,"85,231"


In [15]:
#save dfs as csvs at the end of a session

# this only needs to be called if mojo is scraped again 

# bo_1000_clean_df.to_csv('bo_1000_clean_df.csv', index=False)

In [35]:
df.to_csv('df.csv', index=False)
financial_df.to_csv('financial_df.csv', index=False)
non_english_df.to_csv('non_english_df.csv', index=False)
attributes_df.to_csv('attributes_df.csv', index=False)

with open('omdb_attrs.json', 'w+') as e:
    json.dump(omdb_attrs, e)

In [4]:
#makes list 1,000 dictionaries, each containing the name and revenue of the top 100 films for last 10 years
#scrapes the boxofficemojo website, save result as csv and reuse as computationally expensive and
#don't want to risk getting blacklisted

def box_office_1000(url, year):
    col_names=['rank', 'title', 'worldwide $', 'domestic $', 'domestic %', 'foreign $', 'foreign %']
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    container = soup.find('div', class_='a-section imdb-scroll-table-inner')
    tables = container.find('table').findAll('tr')
    for f in range(1,101):
        time.sleep(random.uniform(1, 3))
        film_row = tables[f]
        cols = film_row.findAll('td')
        row_dict={}
        row_dict['year']=year
        c=0
        for col in cols:
            col_name = col_names[c]
            row_dict[col_name] = col.text
            c+=1
        box_office_list.append(row_dict)

        

In [5]:
# triggers the scraper and makes a dataframe of the 1000 results

box_office_list = []

def next_year():
    for d in range(10):
        year = 2021 - d
        string_year = str(year)
        base_url = 'https://www.boxofficemojo.com/year/world/'
        url = base_url + string_year + "/"
        box_office_1000(url, string_year)    
        
next_year()
box_office_1000_df = pd.DataFrame(box_office_list)
box_office_1000_df

,year,rank,title,worldwide $,domestic $,domestic %,foreign $,foreign %
0,2021,1,"Hi, Mom","$822,009,764",-,-,"$822,009,764",100%
1,2021,2,F9: The Fast Saga,"$714,398,815","$172,931,815",24.2%,"$541,467,000",75.8%
2,2021,3,Detective Chinatown 3,"$686,257,563",-,-,"$686,257,563",100%
3,2021,4,Godzilla vs. Kong,"$467,863,133","$100,563,133",21.5%,"$367,300,000",78.5%
4,2021,5,Black Widow,"$377,665,240","$183,177,333",48.5%,"$194,487,907",51.5%
...,...,...,...,...,...,...,...,...
995,2012,96,The Watch,"$68,267,862","$35,353,000",51.8%,"$32,914,862",48.2%
996,2012,97,Moonrise Kingdom,"$68,263,166","$45,512,466",66.7%,"$22,750,700",33.3%
997,2012,98,The Secret of the Gods,"$67,537,798",-,-,"$67,537,798",100%
998,2012,99,Madea's Witness Protection,"$66,899,242","$65,653,242",98.1%,"$1,246,000",1.9%


In [49]:
# strip $  , and % from values that need to be floats

bo_1000_clean_df = box_office_1000_df.applymap(lambda x: x[1:].replace(',', '') if  x[0] == "$" else x)
bo_1000_clean_df = bo_1000_clean_df.applymap(lambda x: x[:-1] if  x[-1] == "%" else x)
bo_1000_clean_df

,year,rank,title,worldwide $,domestic $,domestic %,foreign $,foreign %
0,2021,1,"Hi, Mom",822009764,-,-,822009764,100
1,2021,2,F9: The Fast Saga,714398815,172931815,24.2,541467000,75.8
2,2021,3,Detective Chinatown 3,686257563,-,-,686257563,100
3,2021,4,Godzilla vs. Kong,467863133,100563133,21.5,367300000,78.5
4,2021,5,Black Widow,377665240,183177333,48.5,194487907,51.5
...,...,...,...,...,...,...,...,...
995,2012,96,The Watch,68267862,35353000,51.8,32914862,48.2
996,2012,97,Moonrise Kingdom,68263166,45512466,66.7,22750700,33.3
997,2012,98,The Secret of the Gods,67537798,-,-,67537798,100
998,2012,99,Madea's Witness Protection,66899242,65653242,98.1,1246000,1.9


In [50]:
# replace - with 0 to format for converting to float

bo_1000_clean_df = bo_1000_clean_df.applymap(lambda x: '0' if  x == "-" else x)
bo_1000_clean_df.head(1000)

,year,rank,title,worldwide $,domestic $,domestic %,foreign $,foreign %
0,2021,1,"Hi, Mom",822009764,0,0,822009764,100
1,2021,2,F9: The Fast Saga,714398815,172931815,24.2,541467000,75.8
2,2021,3,Detective Chinatown 3,686257563,0,0,686257563,100
3,2021,4,Godzilla vs. Kong,467863133,100563133,21.5,367300000,78.5
4,2021,5,Black Widow,377665240,183177333,48.5,194487907,51.5
...,...,...,...,...,...,...,...,...
995,2012,96,The Watch,68267862,35353000,51.8,32914862,48.2
996,2012,97,Moonrise Kingdom,68263166,45512466,66.7,22750700,33.3
997,2012,98,The Secret of the Gods,67537798,0,0,67537798,100
998,2012,99,Madea's Witness Protection,66899242,65653242,98.1,1246000,1.9


In [54]:
# get rid of < beofore any of the values to turn into float - only occurs on <0.1 instances, so can call 0.1

bo_1000_clean_df[['domestic %', 'foreign %']] = bo_1000_clean_df[['domestic %', 'foreign %']].applymap(lambda x: x[1:] if x[0] == '<' else x)
bo_1000_clean_df.head(1000)

,year,rank,title,worldwide $,domestic $,domestic %,foreign $,foreign %
0,2021,1,"Hi, Mom",822009764,0,0,822009764,100
1,2021,2,F9: The Fast Saga,714398815,172931815,24.2,541467000,75.8
2,2021,3,Detective Chinatown 3,686257563,0,0,686257563,100
3,2021,4,Godzilla vs. Kong,467863133,100563133,21.5,367300000,78.5
4,2021,5,Black Widow,377665240,183177333,48.5,194487907,51.5
...,...,...,...,...,...,...,...,...
995,2012,96,The Watch,68267862,35353000,51.8,32914862,48.2
996,2012,97,Moonrise Kingdom,68263166,45512466,66.7,22750700,33.3
997,2012,98,The Secret of the Gods,67537798,0,0,67537798,100
998,2012,99,Madea's Witness Protection,66899242,65653242,98.1,1246000,1.9


In [56]:
#convert numerical columns to floats
bo_1000_clean_df[['worldwide $', 'domestic $', 'foreign $', 'domestic %', 'foreign %']] = bo_1000_clean_df[['worldwide $', 'domestic $', 'foreign $', 'domestic %', 'foreign %']].astype(float)
bo_1000_clean_df

,year,rank,title,worldwide $,domestic $,domestic %,foreign $,foreign %
0,2021,1,"Hi, Mom",822009764.0,0.0,0.0,822009764.0,100.0
1,2021,2,F9: The Fast Saga,714398815.0,172931815.0,24.2,541467000.0,75.8
2,2021,3,Detective Chinatown 3,686257563.0,0.0,0.0,686257563.0,100.0
3,2021,4,Godzilla vs. Kong,467863133.0,100563133.0,21.5,367300000.0,78.5
4,2021,5,Black Widow,377665240.0,183177333.0,48.5,194487907.0,51.5
...,...,...,...,...,...,...,...,...
995,2012,96,The Watch,68267862.0,35353000.0,51.8,32914862.0,48.2
996,2012,97,Moonrise Kingdom,68263166.0,45512466.0,66.7,22750700.0,33.3
997,2012,98,The Secret of the Gods,67537798.0,0.0,0.0,67537798.0,100.0
998,2012,99,Madea's Witness Protection,66899242.0,65653242.0,98.1,1246000.0,1.9


In [4]:
df = df[['title', 'year', 'worldwide $', 'domestic $', 'domestic %', 'foreign $', 'foreign %']]

In [5]:
# a lot of films in the list are not in English, and are local releases in big
# markets. We will look at top performing English lang films in these markets,
# but trying to compete with local production companies for individual foreign
# markets isn't worthwhile.
# So we will split the list into English and non-English speaking movies. 
# This isn't so easy because the film names for foreign language films are 
# often not the same in boxoffice mojo and imdb (eg, top 2021 release 'Hi, Mom')
# is called Ni hao, Li Huan Ying in imdb. It is therefore difficult to access the 
# metadata through imdb and filter by language
# an easier and fairly reliable solution id to filter out by films where 95%
# of the revenues were made in countries outside the US
#(manual check through films showed all foreign language)
# this created two dfs for english and non-english movies

# the english df will be a df showing box office performance od eng lang films
# will end up with another df that holds the attributes of the same films, which will be 
# cross referenced when exploring the data

# so we will call this financial dataframe

financial_df = df[df['domestic %'] >= 5]
non_english_df = df[df['domestic %'] <= 5]



In [6]:
# need to create new column to hold the imdb code, it will be the primary key and used to 
#access other databases. Can't use title as not always unique and can be formatting 
# differences between different databases. Imbd code is unique
# set a default so it won't break when were are using the code to construct api urls

financial_df.insert(4, "imdb_code", "No_code")

In [7]:
financial_df = financial_df[['imdb_code', 'title', 'year', 'worldwide $', 'domestic $', 'domestic %', 'foreign $', 'foreign %']]
financial_df.head()

,imdb_code,title,year,worldwide $,domestic $,domestic %,foreign $,foreign %
1,No_code,F9: The Fast Saga,2021,714398815.0,172931815.0,24.2,541467000.0,75.8
3,No_code,Godzilla vs. Kong,2021,467863133.0,100563133.0,21.5,367300000.0,78.5
4,No_code,Black Widow,2021,377665240.0,183177333.0,48.5,194487907.0,51.5
5,No_code,A Quiet Place Part II,2021,297372261.0,160072261.0,53.8,137300000.0,46.2
6,No_code,Free Guy,2021,277748887.0,102882486.0,37.0,174866401.0,63.0


In [23]:
# this goes through each of the 724 eng lang films and retrieves its data from
# omdb. the imdb api was down and omdb has similar data, with high call limit
# it then creates a dictionary of dictionaries for each film, holding values for 
# attributes that are likely to influence the business proposals
# it also uses the imdbid key to update the imdb_id column in financial df,
# giving us a primary key

omdb_attrs = {}
dud_url=[]
retrieve_error=[]

def on_success(movie_data, financial_df, m):
    imdb_id = movie_data['imdbID']  
    financial_df.loc[financial_df.index[m], 'imdb_code'] = imdb_id
    keys = ['Actors', 'Director', 'Genre', 'Plot', 'Rated', 'Ratings', 'Runtime', 'Writer', 'Title', 'imdbRating', 'imdbVotes']
    omdb_attrs[imdb_id] = {x:movie_data[x] for x in keys}

def url_maker(year, title):
    url = 'http://www.omdbapi.com/?t=' + title + '&y=' + year + '&apikey=' + omdb_key
    return url

#manual correction of alternative titles
def title_corrector(title):
    if title == 'F9: The Fast Saga':
        title = 'F9'
    return title

def get_page(url):
    response=requests.get(url)
    movie_data = json.loads(response.content.decode('utf-8')) 
    return movie_data


def make_omdb_dict(financial_df):
    for m in range(723):
        movie = financial_df.iloc[m,1]
        title = movie.replace(',', '%2C').replace (' ', '+').replace (':', '%3A')
        year = str(financial_df.iloc[m,2])
        url = url_maker(year, title)
        try:
            movie_data = get_page(url)
            if "Error" in movie_data:  
                prev_year = (str(int(year) -1))
                url = url_maker(prev_year, title)
                movie_data = get_page(url)
                if "Error" in movie_data:
                    alt_title = title_corrector(movie)
                    url = url_maker(year, alt_title)
                    movie_data = get_page(url)
                    
                    #used during dev to check for non returning films
                    if "Error" in movie_data:
                        film_error = movie + " - " + movie_data_t['Error'] + "\n"
                        retrieve_error.append(prev_film_error)
                    else:
                        on_success(movie_data, financial_df, m)
                else:
                    on_success(movie_data, financial_df, m)  
            else:
                on_success(movie_data, financial_df, m)
        except:
            dud_url.append(movie)

make_omdb_dict(financial_df)


In [24]:
# check films where imdb id not found
# can discard these as they are either reissues or foreign language films
financial_df.title[financial_df.imdb_code == 'No_code']

43                             Demon Slayer: Mugen Train
203                             Dragon Ball Super: Broly
262                          Baahubali 2: The Conclusion
290                                         The Commuter
329                                The Angry Birds Movie
339    Teenage Mutant Ninja Turtles 2: Out of the Sha...
413                      The Divergent Series: Insurgent
438                                         Ride Along 2
442                                         The 5th Wave
546                                         Crimson Peak
588                              Lee Daniels' The Butler
603                        Jurassic Park 2013 Re-release
608                            Instructions Not Included
625                                        Scary Movie V
654                              Titanic 2012 3D Release
664                                    We're the Millers
686                                           Last Vegas
692                            

In [28]:
# explode the ratings nested dictionary to retrieve the rtomatoes and metacritic
# scores. Imdb ratings already there

for key in omdb_attrs:
    for k in omdb_attrs[key]['Ratings']:
        outlet_ratings = list(k.values())
        if outlet_ratings[0] == 'Rotten Tomatoes':
            omdb_attrs[key]['RottenRating'] = outlet_ratings[1][:-1]
        elif outlet_ratings[0] == 'Metacritic':
            omdb_attrs[key]['MetacriticRating'] = outlet_ratings[1][:-4]


In [33]:
# make the attributes df from the dictionary

attributes_df = pd.DataFrame(omdb_attrs).transpose()
attributes_df.columns = ['Actors', 'Director', 'Genre', 'Plot', 'Rated', 'Ratings', 'Runtime', 'Writer', 'Title', 'imdbRating', 'imdbVotes', 'RottenRating', 'MetacriticRating']



In [34]:
attributes_df = attributes_df[['Title', 'Actors', 'Director', 'Genre', 'Plot', 'Rated', 'Runtime', 'Writer', 'RottenRating', 'MetacriticRating', 'imdbRating', 'imdbVotes']]
attributes_df.reset_index(inplace=True)
attributes_df = attributes_df.rename(columns = {'index':'imdb_code'})
attributes_df.head(3)

,imdb_code,Title,Actors,Director,Genre,Plot,Rated,Runtime,Writer,RottenRating,MetacriticRating,imdbRating,imdbVotes
0,tt5433138,F9,"Vin Diesel, Michelle Rodriguez, Jordana Brewster",Justin Lin,"Action, Adventure, Crime",Dom and the crew must take on an international...,PG-13,143 min,"Daniel Casey, Justin Lin, Alfredo Botello",59,58,5.2,"85,231"
1,tt5034838,Godzilla vs. Kong,"Alexander Skarsgård, Millie Bobby Brown, Rebec...",Adam Wingard,"Action, Sci-Fi, Thriller",The epic next chapter in the cinematic Monster...,PG-13,113 min,"Terry Rossio, Michael Dougherty, Zach Shields",75,59,6.4,"166,316"
2,tt3480822,Black Widow,"Scarlett Johansson, Florence Pugh, David Harbour",Cate Shortland,"Action, Adventure, Sci-Fi",Natasha Romanoff confronts the darker parts of...,PG-13,134 min,"Eric Pearson, Jac Schaeffer, Ned Benson",79,67,6.8,"187,336"


In [ ]:
#trying to find budgets to attach to the financials table....
#imdb has a budget reposnse in api, but many are missing.
# used this code to check before deciding whether to use this api

def budget_scraper(imdb_id):
    try:
        url = 'https://imdb-api.com/en/API/Title/' + imdb_key + '/' + imdb_id
        imdb_data = json.loads(requests.get(url).content.decode('utf-8'))
        budget_data = imdb_data['boxOffice']['budget']
        print(budget_data)
        budget = budget_data.split()
        if budget:
            return budget
        else: 
            budget = 'no_budget'
            return budget
    except:
        budget='no budget'

def add_budget_to_fin_df():
    for index, row in financial_df.iterrows():
        budget = budget_scraper(row['imdb_code'])

add_budget_to_fin_df()

In [89]:
#the movie db also has budget response. Used this to check (have to get internal id
# from imdbid and then search for movie using that)
# returned default budget for all films, so discarded

def budget_scraper_tmd(imdb_id):
    try:
        tmdb_movie_url = 'https://api.themoviedb.org/3/find/' + imdb_id + '?api_key=' + tmdb_key + '&language=en-US&external_source=imdb_id'
        tmdb_id_data = requests.get(tmdb_movie_url)
        tm_id = str(json.loads(tmdb_id_data.content.decode('utf-8'))['movie_results'][0]['id'])
        get_movie_url = 'https://api.themoviedb.org/3/movie/' + tm_id + '?api_key=' + tmdb_key + '&language=en-US'
        tmdb_data = json.loads(requests.get(tm_url).content.decode('utf-8'))
        budget = tmdb_data['budget']
        return budget
    except:
        budget = 'not known'
        return budget

def add_budget_to_fin_df_tmd():
    for index, row in financial_df.iterrows():
        budget = budget_scraper_tmd(row['imdb_code'])
        print(budget)
            
add_budget_to_fin_df_tmd()

200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
not known
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
not known
200000000
200000000
200000000
200000000
200000000
not known
200000000
200000000
200000000
200000000
200000000
not known
200000000
200000000
not known
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
not known
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000
200000000


In [ ]:
print('dog')